In [2]:
import pandas as pd
pd.set_option('display.max_columns', 20)
# pd.set_option('display.max_rows', 20)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
from sklearn.preprocessing import MinMaxScaler

In [6]:
df_ = pd.read_excel("online_retail_II.xlsx",
                    sheet_name="Year 2010-2011")

In [7]:
df = df_.copy()

In [8]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55000,17850.00000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75000,17850.00000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom


In [9]:
df = df[~df["Invoice"].str.contains("C", na=False)]
df = df[(df['Quantity'] > 0)]
df.dropna(inplace=True)
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [12]:
cltv_df = df.groupby('Customer ID').agg({'Invoice': lambda x: len(x),
                                         'Quantity': lambda x: x.sum(),
                                         'TotalPrice': lambda x: x.sum()})

In [13]:
cltv_df.columns = ['total_transaction', 'total_unit', 'total_price']

In [14]:
cltv_df.head()

,total_transaction,total_unit,total_price
Customer ID,,,
12346.00000,1,74215,77183.60000
12347.00000,182,2458,4310.00000
12348.00000,31,2341,1797.24000
12349.00000,73,631,1757.55000
12350.00000,17,197,334.40000


In [ ]:
# CLTV = (Customer_Value / Churn_Rate) x Profit_margin.
# Customer_Value = Average_Order_Value * Purchase_Frequency
# Average_Order_Value = Total_Revenue / Total_Number_of_Orders
# Purchase_Frequency =  Total_Number_of_Orders / Total_Number_of_Customers
# Churn_Rate = 1 - Repeat_Rate

In [15]:
cltv_df['avg_order_value'] = cltv_df['total_price'] / cltv_df['total_transaction']

In [16]:
cltv_df["purchase_frequency"] = cltv_df['total_transaction'] / cltv_df.shape[0]

In [17]:
repeat_rate = cltv_df[cltv_df.total_transaction > 1].shape[0] / cltv_df.shape[0]

In [18]:
churn_rate = 1 - repeat_rate

In [19]:
cltv_df['profit_margin'] = cltv_df['total_price'] * 0.05

In [20]:
cltv_df.head()

,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin
Customer ID,,,,,,
12346.00000,1,74215,77183.60000,77183.60000,0.00023,3859.18000
12347.00000,182,2458,4310.00000,23.68132,0.04195,215.50000
12348.00000,31,2341,1797.24000,57.97548,0.00714,89.86200
12349.00000,73,631,1757.55000,24.07603,0.01682,87.87750
12350.00000,17,197,334.40000,19.67059,0.00392,16.72000


In [21]:
cltv_df['CV'] = (cltv_df['avg_order_value'] * cltv_df["purchase_frequency"]) / churn_rate

In [22]:
cltv_df['CLTV'] = cltv_df['CV'] * cltv_df['profit_margin']

In [23]:
cltv_df.sort_values("CLTV", ascending=False)

,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin,CV,CLTV
Customer ID,,,,,,,,
14646.00000,2080,197491,280206.02000,134.71443,0.47937,14010.30100,3891.75028,54524592.80850
18102.00000,431,64124,259657.30000,602.45313,0.09933,12982.86500,3606.35139,46820773.22451
17450.00000,337,69993,194550.79000,577.30205,0.07767,9727.53950,2702.09431,26284729.09002
16446.00000,3,80997,168472.50000,56157.50000,0.00069,8423.62500,2339.89583,19710405.03906
14911.00000,5677,80515,143825.06000,25.33469,1.30837,7191.25300,1997.57028,14365033.25278
...,...,...,...,...,...,...,...,...
17956.00000,1,1,12.75000,12.75000,0.00023,0.63750,0.17708,0.11289
16454.00000,2,3,6.90000,3.45000,0.00046,0.34500,0.09583,0.03306
14792.00000,2,2,6.20000,3.10000,0.00046,0.31000,0.08611,0.02669


In [24]:
scaler = MinMaxScaler(feature_range=(1, 100))
scaler.fit(cltv_df[["CLTV"]])
cltv_df["SCALED_CLTV"] = scaler.transform(cltv_df[["CLTV"]])

In [25]:
cltv_df.sort_values("CLTV", ascending=False)

,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin,CV,CLTV,SCALED_CLTV
Customer ID,,,,,,,,,
14646.00000,2080,197491,280206.02000,134.71443,0.47937,14010.30100,3891.75028,54524592.80850,100.00000
18102.00000,431,64124,259657.30000,602.45313,0.09933,12982.86500,3606.35139,46820773.22451,86.01222
17450.00000,337,69993,194550.79000,577.30205,0.07767,9727.53950,2702.09431,26284729.09002,48.72504
16446.00000,3,80997,168472.50000,56157.50000,0.00069,8423.62500,2339.89583,19710405.03906,36.78807
14911.00000,5677,80515,143825.06000,25.33469,1.30837,7191.25300,1997.57028,14365033.25278,27.08251
...,...,...,...,...,...,...,...,...,...
17956.00000,1,1,12.75000,12.75000,0.00023,0.63750,0.17708,0.11289,1.00000
16454.00000,2,3,6.90000,3.45000,0.00046,0.34500,0.09583,0.03306,1.00000
14792.00000,2,2,6.20000,3.10000,0.00046,0.31000,0.08611,0.02669,1.00000


In [27]:
cltv_df[["total_transaction", "total_unit", "total_price", "CLTV", "SCALED_CLTV"]].sort_values(by="SCALED_CLTV",
                                                                                               ascending=False).head()

,total_transaction,total_unit,total_price,CLTV,SCALED_CLTV
Customer ID,,,,,
14646.00000,2080,197491,280206.02000,54524592.80850,100.00000
18102.00000,431,64124,259657.30000,46820773.22451,86.01222
17450.00000,337,69993,194550.79000,26284729.09002,48.72504
16446.00000,3,80997,168472.50000,19710405.03906,36.78807
14911.00000,5677,80515,143825.06000,14365033.25278,27.08251


In [28]:
cltv_df.sort_values("total_price", ascending=False)

,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin,CV,CLTV,SCALED_CLTV
Customer ID,,,,,,,,,
14646.00000,2080,197491,280206.02000,134.71443,0.47937,14010.30100,3891.75028,54524592.80850,100.00000
18102.00000,431,64124,259657.30000,602.45313,0.09933,12982.86500,3606.35139,46820773.22451,86.01222
17450.00000,337,69993,194550.79000,577.30205,0.07767,9727.53950,2702.09431,26284729.09002,48.72504
16446.00000,3,80997,168472.50000,56157.50000,0.00069,8423.62500,2339.89583,19710405.03906,36.78807
14911.00000,5677,80515,143825.06000,25.33469,1.30837,7191.25300,1997.57028,14365033.25278,27.08251
...,...,...,...,...,...,...,...,...,...
17956.00000,1,1,12.75000,12.75000,0.00023,0.63750,0.17708,0.11289,1.00000
16454.00000,2,3,6.90000,3.45000,0.00046,0.34500,0.09583,0.03306,1.00000
14792.00000,2,2,6.20000,3.10000,0.00046,0.31000,0.08611,0.02669,1.00000


In [29]:
cltv_df["segment"] = pd.qcut(cltv_df["SCALED_CLTV"], 4, labels=["D", "C", "B", "A"])

In [30]:
cltv_df[["segment", "total_transaction", "total_unit", "total_price", "CLTV", "SCALED_CLTV"]].sort_values(
    by="SCALED_CLTV",
    ascending=False).head()

,segment,total_transaction,total_unit,total_price,CLTV,SCALED_CLTV
Customer ID,,,,,,
14646.00000,A,2080,197491,280206.02000,54524592.80850,100.00000
18102.00000,A,431,64124,259657.30000,46820773.22451,86.01222
17450.00000,A,337,69993,194550.79000,26284729.09002,48.72504
16446.00000,A,3,80997,168472.50000,19710405.03906,36.78807
14911.00000,A,5677,80515,143825.06000,14365033.25278,27.08251


In [31]:
cltv_df.groupby("segment")[["total_transaction", "total_unit", "total_price", "CLTV", "SCALED_CLTV"]].agg(
    {"count", "mean", "sum"})

total_transaction                 total_unit                   \
                      sum count      mean        sum count       mean   
segment                                                                 
D                   18682  1085  17.21843     140493  1085  129.48664   
C                   40122  1085  36.97880     310632  1085  286.29677   
B                   82856  1084  76.43542     703995  1084  649.44188   
A                  256265  1085 236.18894    4026577  1085 3711.13088   

          total_price                             CLTV                     \
                  sum count       mean             sum count         mean   
segment                                                                     
D        194130.09000  1085  178.92174     28106.99627  1085     25.90507   
C        503836.85200  1085  464.36576    171620.23375  1085    158.17533   
B       1161913.31100  1084 1071.87575    924739.47491  1084    853.08070   
A       7051545.65100  1085 6499.12042 254961091.91196  1085 234987.18149   

        SCALED_CLTV                
                sum count    mean  
segment                            
D        1085.05103  1085 1.00005  
C        1085.31161  1085 1.00029  
B        1085.67904  1084 1.00155  
A        1547.93144  1085 1.42666